In [37]:
!pip install prefixspan

In [38]:
from prefixspan import PrefixSpan
from collections import defaultdict
from datetime import datetime

# Padrões de sequência - Exercício
| IdCliente | Itens   | Data  |
| --------- | ------- | ----- |
| 1         | A, B    | 10/09 |
| 2         | B       | 11/09 |
| 1         | D, E    | 12/09 |
| 3         | A, D, E | 13/09 |
| 1         | A, B, C | 14/09 |
| 2         | C, D, E | 15/09 |


In [39]:
data = [
    (1, ['A', 'B'], '10/09'),
    (2, ['B'], '11/09'),
    (1, ['D', 'E'], '12/09'),
    (3, ['A', 'D', 'E'], '13/09'),
    (1, ['A', 'B', 'C'], '14/09'),
    (2, ['C', 'D', 'E'], '15/09'),
]

## Usando o PrefixSpan

In [40]:
# Função para converter a data no formato 'dia/mês' para um objeto datetime (ano fixo não importa aqui)
def parse_date(d):
    return datetime.strptime(d, '%d/%m')

# Cria um dicionário onde a chave é o cliente e o valor é uma lista (sequência) dos itens consumidos
cliente_seq = defaultdict(list)

# Ordena os dados primeiro pelo cliente e depois pela data (para garantir a ordem temporal das transações)
for cliente, itens, data_str in sorted(data, key=lambda x: (x[0], parse_date(x[2]))):
    # Para cada item na lista de itens do evento, adiciona o item individualmente na sequência do cliente
    # Isso transforma cada itemset em eventos únicos, respeitando a ordem temporal
    for item in itens:
        cliente_seq[cliente].append(item)

# Cria uma lista com as sequências de cada cliente, no formato esperado pelo PrefixSpan
sequences = list(cliente_seq.values())

# Instancia o algoritmo PrefixSpan com as sequências
ps = PrefixSpan(sequences)

# Executa a mineração de sequências frequentes com suporte mínimo igual a 2 (ou seja, presente em pelo menos 2 dos 3 clientes)
frequent_seqs = ps.frequent(minsup=2)

# Imprime os padrões encontrados, ordenando primeiro pelo suporte decrescente e depois pela sequência
print("🔹 Padrões de sequência frequentes (PrefixSpan) com eventos únicos:")
for support, sequence in sorted(frequent_seqs, key=lambda x: (-x[0], x[1])):
    print(f"Sequência: {sequence}, Suporte: {support}")

🔹 Padrões de sequência frequentes (PrefixSpan) com eventos únicos:
Sequência: ['D'], Suporte: 3
Sequência: ['D', 'E'], Suporte: 3
Sequência: ['E'], Suporte: 3
Sequência: ['A'], Suporte: 2
Sequência: ['A', 'D'], Suporte: 2
Sequência: ['A', 'D', 'E'], Suporte: 2
Sequência: ['A', 'E'], Suporte: 2
Sequência: ['B'], Suporte: 2
Sequência: ['B', 'C'], Suporte: 2
Sequência: ['B', 'D'], Suporte: 2
Sequência: ['B', 'D', 'E'], Suporte: 2
Sequência: ['B', 'E'], Suporte: 2
Sequência: ['C'], Suporte: 2


## Usando o AprioriAll

In [41]:
def apriori_all(data, min_support):
    # Fase 1: Ordenação
    sorted_data = sorted(data, key=lambda x: (x[0], x[2]))

    # Fase 2: Criar sequências de consumidores
    customer_sequences = defaultdict(list)
    for customer, items, date in sorted_data:
        customer_sequences[customer].append(set(items))

    # Converter para lista de sequências
    sequences = [list(seq) for seq in customer_sequences.values()]
    total_customers = len(sequences)

    # Fase 3: Encontrar itemsets frequentes
    def find_frequent_itemsets(sequences, min_support):
        itemsets = defaultdict(int)

        # Contar todos os itemsets de tamanho 1
        for seq in sequences:
            for itemset in seq:
                for item in itemset:
                    itemsets[frozenset([item])] += 1

        # Gerar itemsets candidatos de tamanho > 1
        k = 2
        frequent_itemsets = {}
        current_frequent = {itemset: count for itemset, count in itemsets.items()
                           if count / total_customers >= min_support}

        while current_frequent:
            frequent_itemsets[k-1] = current_frequent
            candidates = set()

            # Gerar candidatos combinando itemsets frequentes de tamanho k-1
            itemset_list = list(current_frequent.keys())
            for i in range(len(itemset_list)):
                for j in range(i+1, len(itemset_list)):
                    new_candidate = itemset_list[i].union(itemset_list[j])
                    if len(new_candidate) == k:
                        candidates.add(new_candidate)

            # Contar suporte dos candidatos
            candidate_counts = defaultdict(int)
            for seq in sequences:
                for itemset in seq:
                    for candidate in candidates:
                        if candidate.issubset(itemset):
                            candidate_counts[candidate] += 1

            # Filtrar pelos que atendem ao suporte mínimo
            current_frequent = {itemset: count for itemset, count in candidate_counts.items()
                              if count / total_customers >= min_support}
            k += 1

        return frequent_itemsets

    frequent_itemsets = find_frequent_itemsets(sequences, min_support)

    # Mapear itemsets frequentes para IDs
    itemset_to_id = {}
    id_to_itemset = {}
    id_counter = 1
    for k in frequent_itemsets:
        for itemset in frequent_itemsets[k]:
            itemset_to_id[itemset] = id_counter
            id_to_itemset[id_counter] = itemset
            id_counter += 1

    # Fase 4: Transformação - substituir itemsets pelos IDs mapeados
    transformed_sequences = []
    for seq in sequences:
        transformed_seq = []
        for itemset in seq:
            transformed_itemset = set()
            for fi in itemset_to_id:
                if fi.issubset(itemset):
                    transformed_itemset.add(itemset_to_id[fi])
            if transformed_itemset:
                transformed_seq.append(transformed_itemset)
        transformed_sequences.append(transformed_seq)

    # Fase 5: Mineração de sequências frequentes
    def find_frequent_sequences(transformed_sequences, min_support):
        # Inicializar com 1-sequências (itemsets frequentes)
        F = {}
        F[1] = defaultdict(int)

        for seq in transformed_sequences:
            for itemset in seq:
                for item in itemset:
                    F[1][(item,)] += 1

        F[1] = {seq: count for seq, count in F[1].items()
                if count / total_customers >= min_support}

        k = 2
        while k-1 in F and F[k-1]:
            # Gerar candidatos
            candidates = set()
            for seq1 in F[k-1]:
                for seq2 in F[k-1]:
                    # Junção: os primeiros k-2 elementos devem ser iguais
                    if k == 2 or seq1[:-1] == seq2[:-1]:
                        # Caso 1: adicionar último elemento de seq2 como novo itemset
                        new_seq = seq1 + (seq2[-1],)
                        candidates.add(new_seq)
                        # Caso 2: combinar último itemset (se possível)
                        if k > 2 and seq1[-2] == seq2[-2]:
                            combined_last = (seq1[-1], seq2[-1])
                            new_seq = seq1[:-1] + combined_last
                            candidates.add(new_seq)

            # Poda: verificar se todas as subsequências de tamanho k-1 são frequentes
            pruned_candidates = set()
            for candidate in candidates:
                valid = True
                # Gerar todas as subsequências de tamanho k-1
                for i in range(len(candidate)):
                    if k == 2:
                        subseq = (candidate[i],)
                    else:
                        subseq = candidate[:i] + candidate[i+1:]
                    if subseq not in F[k-1]:
                        valid = False
                        break
                if valid:
                    pruned_candidates.add(candidate)

            # Contar suporte dos candidatos podados
            candidate_counts = defaultdict(int)
            for candidate in pruned_candidates:  # Corrigido: usar a variável definida
                for seq in transformed_sequences:
                    # Verificar se a sequência contém o candidato (em ordem)
                    i = 0
                    for itemset in seq:
                        if i < len(candidate) and candidate[i] in itemset:
                            i += 1
                        if i == len(candidate):
                            break
                    if i == len(candidate):
                        candidate_counts[candidate] += 1

            F[k] = {seq: count for seq, count in candidate_counts.items()
                    if count / total_customers >= min_support}
            k += 1

        return F

    frequent_sequences = find_frequent_sequences(transformed_sequences, min_support)

    # Converter IDs de volta para itemsets
    final_sequences = {}
    for k in frequent_sequences:
        final_sequences[k] = {}
        for seq, count in frequent_sequences[k].items():
            decoded_seq = []
            for item in seq:
                decoded_seq.append(id_to_itemset[item])
            final_sequences[k][tuple(decoded_seq)] = count

    return final_sequences

# Executar o algoritmo com suporte mínimo de 50% (0.5)
result = apriori_all(data, min_support=0.5)

# Exibir os resultados
for k in result:
    print(f"\nSequências frequentes de tamanho {k}:")
    for seq, count in result[k].items():
        print(f"{seq}: suporte = {count}/3 = {count/3:.2f}")


Sequências frequentes de tamanho 1:
(frozenset({'A'}),): suporte = 3/3 = 1.00
(frozenset({'B'}),): suporte = 3/3 = 1.00
(frozenset({'A', 'B'}),): suporte = 2/3 = 0.67
(frozenset({'E'}),): suporte = 3/3 = 1.00
(frozenset({'D'}),): suporte = 3/3 = 1.00
(frozenset({'E', 'D'}),): suporte = 3/3 = 1.00
(frozenset({'C'}),): suporte = 2/3 = 0.67

Sequências frequentes de tamanho 2:
(frozenset({'B'}), frozenset({'C'})): suporte = 2/3 = 0.67
(frozenset({'B'}), frozenset({'D'})): suporte = 2/3 = 0.67
(frozenset({'B'}), frozenset({'E', 'D'})): suporte = 2/3 = 0.67
(frozenset({'B'}), frozenset({'E'})): suporte = 2/3 = 0.67

Sequências frequentes de tamanho 3:
